In [2]:
import snowflake.connector
import requests
import openai
import base64
from openai import OpenAI

In [80]:
# Snowflake credentials
SNOWFLAKE_ACCOUNT=''
SNOWFLAKE_USER=''
SNOWFLAKE_PASSWORD=''
SNOWFLAKE_DATABASE=''
SNOWFLAKE_SCHEMA=''
SNOWFLAKE_SCHEMA1=''
SNOWFLAKE_WAREHOUSE=''
SNOWFLAKE_ROLE=''
SNOWFLAKE_TABLE=''

In [98]:
# Connect to Snowflake
conn1 = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA1
)

In [100]:
cursor1 = conn1.cursor()

In [84]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

# Fetch records from the table
cursor = conn.cursor()
cursor.execute(f"SELECT EVENT_TITLE,IMAGE_URL,START_TIME,END_TIME,LOCATION,FULL_ADDRESS,CATEGORIES,ADMISSION,DESCRIPTION,EVENT_URL FROM {SNOWFLAKE_TABLE}")
records = cursor.fetchall()

In [12]:
def get_image_description(image_url):
    """Download image and get a textual description using OpenAI Vision model"""
    response = requests.get(image_url)
        
        # OpenAI API Call
    client = OpenAI(api_key='')
    response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "You will be provided with an image related to an event. Describe the image concisely, focusing only on event-specific details. If the image includes information such as the event name, timing, date, or price, extract and provide those details. Exclude any unrelated observations or interpretations.",
                },
                {
                    "type": "image_url",
                    "image_url": {"url":  f"{image_url}" },
                },
            ],
        }
    ],
)


    return response.choices[0].message.content

In [28]:
updated_records = []

for record in records:
    title, image_url, start_time, end_time, location, full_address, categories, admission, description, event_url = record


    if image_url is not None and image_url != 'No Image' and image_url.startswith('https'):
        image_description = get_image_description(image_url)
    else:
        image_description = 'No Image associated'
    
    updated_records.append((title, image_url, start_time, end_time, location, full_address, categories, admission, description, event_url, image_description))


In [32]:
updated_records[0]

('V for Vault: An Electronic Dance Music Valentines',
 'https://media.thebostoncalendar.com/images/q_auto,fl_lossy/v1738974124/duz2jd9y4kr285bdamux/v-for-vault-an-electronic-dance-music-valentines.png',
 '2025-02-14 7:00pm EST',
 '2025-02-15 1:00am EST',
 'The Neal Rantoul Vault Theater',
 '25 Exchange St., Lynn, MA 01901',
 'Music, Nightlife, Party, Shows',
 '$15',
 'V for Vault: An Electronic Dance Music Valentine’sGet ready to dance the night away atV for Vault, an unforgettableElectronic Dance Music Valentine’sevent at theNeal Rantoul Vault TheatreonFriday, February 14th!Celebrate love and connection with an electrifying night of beats and bass across genres likebass, club, breaks, DnB, and 140. Whether you’re coming with your Valentine, your squad, or solo, this event promises an energetic vibe that will keep you moving all night long.Event Details:🕖Doors Open:7 PM🎶Lineup:MARY AKMr. EyesLeeD TorranceDINOBLUNT b2b Mung BanditJoin us for this genre-bending night of fusion bass music

In [36]:
import pandas as pd
columns = [
    "Title", "Image URL", "Start Time", "End Time", "Location", "Full Address",
    "Categories", "Admission", "Description", "Event URL", "Image Description"
]

# Create the DataFrame
df = pd.DataFrame(updated_records, columns=columns)

In [38]:
df

,Title,Image URL,Start Time,End Time,Location,Full Address,Categories,Admission,Description,Event URL,Image Description
0,V for Vault: An Electronic Dance Music Valentines,https://media.thebostoncalendar.com/images/q_a...,2025-02-14 7:00pm EST,2025-02-15 1:00am EST,The Neal Rantoul Vault Theater,"25 Exchange St., Lynn, MA 01901","Music, Nightlife, Party, Shows",$15,V for Vault: An Electronic Dance Music Valenti...,https://www.thebostoncalendar.com/events/v-for...,**Event Details:**\n\n- **Event Name:** U FOR ...
1,Artz Underground Presents: For The Love of RnB,https://media.thebostoncalendar.com/images/q_a...,2025-02-15 7:00pm EST,2025-02-16 12:30am EST,The Neal Rantoul Vault Theater,"25 Exchange St., Lynn, MA 01901","Drinks, Music, Nightlife, Shows",$20,"Join us For The Love of RnB on Saturday, Febru...",https://www.thebostoncalendar.com/events/artz-...,**Event Details:**\n\n- **Event Name:** For th...
2,¡Miércoles Maravilloso!: free in-person Spanis...,No Image,2025-02-19 7:15pm EST,2025-02-19 8:15pm EST,Boston Area Spanish Exchange (BASE),"101 Arch St., Boston, MA 02110","Classes, Date Idea, Meetup",FREE,¡Miércoles Maravilloso!Free open-level Spanish...,https://www.thebostoncalendar.com/events/mierc...,No Image associated
3,100 things to do in Boston this weekend,https://media.thebostoncalendar.com/images/q_a...,2025-02-14 7:00am EST,2025-02-17 11:00pm EST,Boston,"Surrounding areas, Boston, MA","Animals, Art, Business & Professional, Classes...",$Varies,Celebrate Valentine’s Day weekend and Presiden...,https://www.thebostoncalendar.com/events/100-t...,The image depicts a skyline at night featuring...
4,91 FREE things to do in Boston this week: Feb ...,https://media.thebostoncalendar.com/images/q_a...,2025-02-10 7:00am EST,2025-02-17 11:00pm EST,Boston,"Surrounding areas, Boston, MA","Animals, Art, Business & Professional, Classes...",FREE,Happy week of love! It’s time to celebrate our...,https://www.thebostoncalendar.com/events/91-fr...,The image depicts a winter scene in an urban a...
...,...,...,...,...,...,...,...,...,...,...,...
1194,Après-ski on the Mystic at The Great American ...,https://media.thebostoncalendar.com/images/q_a...,2025-02-21 8:00pm EST,2025-02-22 12:00am EST,The Great American Beer Hall,"142 Mystic Ave, Medford, MA 02155","Drinks, Nightlife, Outside, Party",$5.00,"Après-ski on the MysticFebruary 21, 2025 8:00 ...",https://www.thebostoncalendar.com/events/apres...,**Event Details:**\n\n- **Event Name:** Après ...
1195,Dirty Disney at Lil Chuck,https://media.thebostoncalendar.com/images/q_a...,2025-02-21 9:30pm EST,2025-02-21 11:00pm EST,Lil Chuck at the Charles Playhouse,"74 Warrenton Street, Boston, MA 02116","Date Idea, Drinks, Good for Groups, Nightlife",$30,"Disney adults rejoice! We're back, baby! Revis...",https://www.thebostoncalendar.com/events/dirty...,"The image features a performance titled ""Dirty..."
1196,$1 Oysters at Bootleg Special,https://media.thebostoncalendar.com/images/q_a...,2025-02-21 10:00pm EST,2025-02-23 1:00am EST,Bootleg Special,"400 Tremont Street, Boston, MA","Date Idea, Food",$1+,Enjoy a late night special of $1 oysters at 10...,https://www.thebostoncalendar.com/events/1-oys...,The image features a beautifully arranged plat...
1197,10PM Detention: Standup Comedy and $4 Drinks,https://media.thebostoncalendar.com/images/q_a...,2025-02-21 10:00pm EST,2025-02-21 11:30pm EST,Goofs Comedy Club,"432 McGrath Highway, Somerville, MA 02143","Accessible Spots, Date Idea, Drinks, Good for ...",$16-22,"Every weekend, we bring our favorite headliner...",https://www.thebostoncalendar.com/events/late-...,The image depicts a comedy event taking place ...


In [40]:
import json

# Function to combine all columns into a single text representation
def combine_row_data(row):
    combined_text = " | ".join([f"{col}: {row[col]}" for col in df.columns])
    return combined_text

# Apply the function to all rows
df["Combined Text"] = df.apply(combine_row_data, axis=1)

In [44]:
def get_embedding(text, cursor):
    query = """
    SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', PARSE_JSON(%s))
    """
    cursor.execute(query, (json.dumps({"text": text}),))
    result = cursor.fetchone()
    return result[0] if result else None

# Generate embeddings for each row
df["Vector Embedding"] = df["Combined Text"].apply(lambda text: get_embedding(text, cursor))


In [46]:
df

,Title,Image URL,Start Time,End Time,Location,Full Address,Categories,Admission,Description,Event URL,Image Description,Combined Text,Vector Embedding
0,V for Vault: An Electronic Dance Music Valentines,https://media.thebostoncalendar.com/images/q_a...,2025-02-14 7:00pm EST,2025-02-15 1:00am EST,The Neal Rantoul Vault Theater,"25 Exchange St., Lynn, MA 01901","Music, Nightlife, Party, Shows",$15,V for Vault: An Electronic Dance Music Valenti...,https://www.thebostoncalendar.com/events/v-for...,**Event Details:**\n\n- **Event Name:** U FOR ...,Title: V for Vault: An Electronic Dance Music ...,"[0.045166015625, 0.0545654296875, 0.0613708496..."
1,Artz Underground Presents: For The Love of RnB,https://media.thebostoncalendar.com/images/q_a...,2025-02-15 7:00pm EST,2025-02-16 12:30am EST,The Neal Rantoul Vault Theater,"25 Exchange St., Lynn, MA 01901","Drinks, Music, Nightlife, Shows",$20,"Join us For The Love of RnB on Saturday, Febru...",https://www.thebostoncalendar.com/events/artz-...,**Event Details:**\n\n- **Event Name:** For th...,Title: Artz Underground Presents: For The Love...,"[0.057525634765625, 0.045196533203125, 0.03808..."
2,¡Miércoles Maravilloso!: free in-person Spanis...,No Image,2025-02-19 7:15pm EST,2025-02-19 8:15pm EST,Boston Area Spanish Exchange (BASE),"101 Arch St., Boston, MA 02110","Classes, Date Idea, Meetup",FREE,¡Miércoles Maravilloso!Free open-level Spanish...,https://www.thebostoncalendar.com/events/mierc...,No Image associated,Title: ¡Miércoles Maravilloso!: free in-person...,"[0.06787109375, -0.041290283203125, 0.01770019..."
3,100 things to do in Boston this weekend,https://media.thebostoncalendar.com/images/q_a...,2025-02-14 7:00am EST,2025-02-17 11:00pm EST,Boston,"Surrounding areas, Boston, MA","Animals, Art, Business & Professional, Classes...",$Varies,Celebrate Valentine’s Day weekend and Presiden...,https://www.thebostoncalendar.com/events/100-t...,The image depicts a skyline at night featuring...,Title: 100 things to do in Boston this weekend...,"[0.06866455078125, 0.0128173828125, 0.03955078..."
4,91 FREE things to do in Boston this week: Feb ...,https://media.thebostoncalendar.com/images/q_a...,2025-02-10 7:00am EST,2025-02-17 11:00pm EST,Boston,"Surrounding areas, Boston, MA","Animals, Art, Business & Professional, Classes...",FREE,Happy week of love! It’s time to celebrate our...,https://www.thebostoncalendar.com/events/91-fr...,The image depicts a winter scene in an urban a...,Title: 91 FREE things to do in Boston this wee...,"[0.0374755859375, 0.007232666015625, 0.0351867..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,Après-ski on the Mystic at The Great American ...,https://media.thebostoncalendar.com/images/q_a...,2025-02-21 8:00pm EST,2025-02-22 12:00am EST,The Great American Beer Hall,"142 Mystic Ave, Medford, MA 02155","Drinks, Nightlife, Outside, Party",$5.00,"Après-ski on the MysticFebruary 21, 2025 8:00 ...",https://www.thebostoncalendar.com/events/apres...,**Event Details:**\n\n- **Event Name:** Après ...,Title: Après-ski on the Mystic at The Great Am...,"[0.093994140625, -0.001102447509765625, 0.0640..."
1195,Dirty Disney at Lil Chuck,https://media.thebostoncalendar.com/images/q_a...,2025-02-21 9:30pm EST,2025-02-21 11:00pm EST,Lil Chuck at the Charles Playhouse,"74 Warrenton Street, Boston, MA 02116","Date Idea, Drinks, Good for Groups, Nightlife",$30,"Disney adults rejoice! We're back, baby! Revis...",https://www.thebostoncalendar.com/events/dirty...,"The image features a performance titled ""Dirty...",Title: Dirty Disney at Lil Chuck | Image URL: ...,"[0.0201416015625, 0.0308380126953125, 0.030563..."
1196,$1 Oysters at Bootleg Special,https://media.thebostoncalendar.com/images/q_a...,2025-02-21 10:00pm EST,2025-02-23 1:00am EST,Bootleg Special,"400 Tremont Street, Boston, MA","Date Idea, Food",$1+,Enjoy a late night special of $1 oysters at 10...,https://www.thebostoncalendar.com/events/1-oys...,The image features a beautifully arranged plat...,Title: 

In [48]:
df.columns

Index(['Title', 'Image URL', 'Start Time', 'End Time', 'Location',
       'Full Address', 'Categories', 'Admission', 'Description', 'Event URL',
       'Image Description', 'Combined Text', 'Vector Embedding'],
      dtype='object')

In [50]:
# Add additional columns
df["source_website"] = "thebostoncalendar.com/"  
df["date_load_time"] = pd.to_datetime("now") 

# Rename columns to match Snowflake schema
df = df.rename(columns={
    "Title": "title",
    "Image URL": "image_url",
    "Start Time": "start_time",
    "End Time": "end_time",
    "Location": "location",
    "Full Address": "full_address",
    "Categories": "categories",
    "Admission": "admission",
    "Description": "description",
    "Event URL": "event_url",
    "Image Description": "image_description",
    "Combined Text": "combined_text",
    "Vector Embedding": "vector_embedding"
})

In [54]:
from snowflake.connector.pandas_tools import write_pandas

In [68]:
# Rename columns to match Snowflake schema with uppercase names
df.columns = df.columns.str.upper()


In [76]:
from snowflake.connector.pandas_tools import write_pandas

result = write_pandas(conn1, df, "FACT_BOSTON_EVENTS_DETAILS")

print(f"Data insertion result: {result}")  


Data insertion result: (True, 1, 1199, [('ixepewalep/file0.txt', 'LOADED', 1199, 1199, 1, 0, None, None, None, None)])


In [102]:
query = """
SELECT
  combined_text,
  VECTOR_COSINE_SIMILARITY(
    vector_embedding::VECTOR(FLOAT, 1024),
    SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', 'Can you suggest me some events related to book purchase near harvard')
  ) AS similarity
FROM fact_boston_events_details
ORDER BY similarity DESC
LIMIT 5;
"""

# Execute query and fetch results
cursor1.execute(query)

In [104]:
# Convert result to a DataFrame
cursor1.fetchall()

[('Title: 11 Book Events in Boston to Help You Jumpstart Your 2025 Reading Goals | Image URL: https://media.thebostoncalendar.com/images/q_auto,fl_lossy/v1736787895/czidf9pqzqirmwsg4tl1/11-book-events-in-boston-to-help-you-jumpstart-your-2025-reading-goals.jpg | Start Time: 2025-01-13 9:00am EST | End Time: 2025-02-15 9:00pm EST | Location: Boston | Full Address: Surrounding areas, Boston, MA  | Categories: Art, Drinks, History, Meetup | Admission: $Varies | Description: Whether you’re looking to get a head start on your 2025 reading goals, explore new genres, or meet other book lovers, these literary happenings around Boston are sure to scratch that reading itch.by Celina Colby1)Papercuts x Loring Greenough: Literary Salon with Marianne Leone & Randy Susan MeyersWhen:Tuesday 1/14Where:Jamaica PlainInfo:See actress and writer Marianne Leone in conversation with local author Randy Susan Meyers at the latest installment of this author talk series created by Papercuts Bookshop and the Lor

In [106]:
# Close cursor and connection
cursor1.close()
conn1.close()